In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
import os

2025-12-11 04:54:31.668740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765428871.875335      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765428871.933671      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
base_path = "/kaggle/input/cmapss1"
print(os.listdir(base_path))

['train_FD001.txt']


In [3]:
def load_and_process_data(filepath):
    print(f"Loading data from {filepath}...")
    
    cols = ['id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f's{i}' for i in range(1, 22)]
    
    data = pd.read_csv(filepath, sep='\s+', header=None, names=cols)
    print(f"Original Data Shape: {data.shape}")

    # RUL
    rul = data.groupby('id')['cycle'].max().reset_index()
    rul.columns = ['id', 'max']
    data = data.merge(rul, on='id', how='left')
    data['RUL'] = data['max'] - data['cycle']
    data.drop('max', axis=1, inplace=True)
    
    print("RUL Calculation Complete.")
    return data

In [4]:
def process_for_training(data, sequence_length=50):
    print("Starting Normalization and Windowing...")
    # Excluding ID, Cycle,RUL 
    train_cols = [col for col in data.columns if col not in ['id', 'cycle', 'RUL']]
    
    # Normalize0-1
    scaler = MinMaxScaler()
    data[train_cols] = scaler.fit_transform(data[train_cols])
    print("Data Normalized.")

    sequences = []
    labels = []
    
    for engine_id in data['id'].unique():
        engine_data = data[data['id'] == engine_id]
        
        features = engine_data[train_cols].values
        target = engine_data['RUL'].values
        
        if len(engine_data) < sequence_length:
            continue
            
        for i in range(len(engine_data) - sequence_length):
            window_data = features[i : i + sequence_length]
            label_value = target[i + sequence_length]
            
            sequences.append(window_data)
            labels.append(label_value)
            
    return np.array(sequences), np.array(labels)

In [5]:
def build_and_train_model(X_train, y_train):
    print("Building LSTM Model...")
    
    model = Sequential([
        # Layer 1: LSTM with 100 neurons
        LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2), 
        
        # Layer 2: LSTM with 50 neurons
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        
        # Layer 3: The Output
        Dense(units=1) 
    ])
    
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae"),tf.keras.metrics.RootMeanSquaredError(name="rmse")])
    
    model.summary()

    print("\nStarting Training... (Go grab a coffee, this takes a few minutes)")
    history = model.fit(
        X_train, y_train,
        epochs=20,      
        batch_size=32,      
        validation_split=0.1, 
        verbose=1
    )
    
    model.save('../models/rnn_model.keras')
    print("\nModel Saved successfully to 'models/rnn_model.keras'!")
    
    return model, history


In [6]:
FILE_PATH = '/kaggle/input/cmapss1/train_FD001.txt' 
    
df = load_and_process_data(FILE_PATH)
    
SEQUENCE_LENGTH = 50
X_train, y_train = process_for_training(df, SEQUENCE_LENGTH)

print("\nSUCCESS! Data is ready for AI.")
print(f"Final Input Shape (X_train): {X_train.shape}") #(Samples, 50, Features)- checkcheck
print(f"Final Target Shape (y_train): {y_train.shape}")

Loading data from /kaggle/input/cmapss1/train_FD001.txt...
Original Data Shape: (20631, 26)
RUL Calculation Complete.
Starting Normalization and Windowing...
Data Normalized.

SUCCESS! Data is ready for AI.
Final Input Shape (X_train): (15631, 50, 24)
Final Target Shape (y_train): (15631,)


In [7]:
model, history = build_and_train_model(X_train, y_train)

Building LSTM Model...


I0000 00:00:1765428886.619008      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 100)        │        50,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,251 (313.48 KB)

...

 Non-trainable params: 0 (0.00 B)


Starting Training... (Go grab a coffee, this takes a few minutes)
Epoch 1/20


I0000 00:00:1765428890.814704      60 cuda_dnn.cc:529] Loaded cuDNN version 90300


440/440 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 8204.0879 - mae: 72.4826 - rmse: 90.5151 - val_loss: 8753.2686 - val_mae: 71.9456 - val_rmse: 93.5589
Epoch 2/20
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 5670.6108 - mae: 57.4215 - rmse: 75.2770 - val_loss: 6727.0063 - val_mae: 62.1619 - val_rmse: 82.0183
Epoch 3/20
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 4252.6055 - mae: 49.6793 - rmse: 65.1949 - val_loss: 5561.7900 - val_mae: 56.9264 - val_rmse: 74.5774
Epoch 4/20
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 3583.2212 - mae: 46.3784 - rmse: 59.8543 - val_loss: 4946.5220 - val_mae: 54.4546 - val_rmse: 70.3315
Epoch 5/20
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 3320.3110 - mae: 45.3002 - rmse: 57.6138 - val_loss: 4651.8438 - val_mae: 53.4511 - val_rmse: 68.2044
Epoch 6/20
440/440 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 3203.6912 - mae: 45.3160 - rmse: 56.5997 - val_loss: 4511.0669 - val_mae: 53.0689 - val_rmse: 67.1645
Epoch 7/20
440/440 ━━━━━━━━━━━━━━━━━━━

FileNotFoundError: [Errno 2] No such file or directory: '../models/rnn_model.keras'